In [11]:
import pandas as pd
import numpy as np
import scipy
import scanpy as sc

import matplotlib.pyplot as plt
import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm
from pathlib import Path

from celloracle import motif_analysis as ma
import celloracle as co
co.__version__


'0.20.0'

In [ ]:
os.chdir('/home/kl467102/thesis/')
dataset_name = "pbmc10k"
cicero_output = Path("results/cicero_output")  / dataset_name
celloracle_dir = Path("results/celloracle_output") / dataset_name
peaks_path = cicero_output / 'peak_gene_list.csv'

### 0. genome preparation

In [4]:
ref_genome = 'hg38'
genome_installation = ma.is_genome_installed(ref_genome=ref_genome,
                                             genomes_dir=None)
print(ref_genome, "installation: ", genome_installation)


hg38 installation:  True


In [5]:
if not genome_installation:
    import genomepy
    genomepy.install_genome(name=ref_genome, provider="UCSC", genomes_dir=None)
else:
    print(ref_genome, "is installed.")


hg38 is installed.


### load data

In [7]:
peaks = pd.read_csv(peaks_path, index_col=0)
peaks.head()


,peak_id,gene_short_name
0,chr10_100009458_100010353,DNMBP
1,chr10_100185609_100186527,ERLIN1
2,chr10_100229090_100229936,CHUK
3,chr10_100267214_100268053,CWF19L1
4,chr10_100286203_100287049,BLOC1S2


In [8]:
peaks = ma.check_peak_format(peaks, ref_genome, genomes_dir=None)


Peaks before filtering:  21256
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  21256


## Instantiate TFinfo object and search for TF binding motifs

In [9]:
tfi = ma.TFinfo(peak_data_frame=peaks, 
                ref_genome=ref_genome,
                genomes_dir=None) 


In [10]:
os.makedirs(celloracle_dir, exist_ok=True)

In [ ]:
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
tfi.scan(fpr=0.02, 
         motifs=None,  # If you enter None, default motifs will be loaded.
         verbose=True)


No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please see https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 



2025-10-11 13:16:53,573 - DEBUG - using background: genome hg38 with size 200


Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 

Motif scan started .. It may take long time.



Scanning:   0%|          | 0/18656 [00:00<?, ? sequences/s]

AttributeError: 'PosixPath' object has no attribute 'endswith'

In [15]:
# Save tfinfo object
hdf5_path = celloracle_dir / "tf.celloracle.tfinfo"
hdf5_path  = str(hdf5_path)
tfi.to_hdf5(file_path=hdf5_path)

In [16]:
tfi.scanned_df.head()


,seqname,motif_id,factors_direct,factors_indirect,score,pos,strand
0,chr10_100009458_100010353,GM.5.0.Mixed.0001,,"EGR1, SRF",8.578792,338,-1
1,chr10_100009458_100010353,GM.5.0.Mixed.0001,,"EGR1, SRF",8.319869,597,-1
2,chr10_100009458_100010353,GM.5.0.Mixed.0001,,"EGR1, SRF",7.919846,705,-1
3,chr10_100009458_100010353,GM.5.0.Mixed.0001,,"EGR1, SRF",7.680395,706,-1
4,chr10_100009458_100010353,GM.5.0.Mixed.0001,,"EGR1, SRF",7.575280,334,-1


## Filtering motifs


In [ ]:
# Reset filtering 
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)


Filtering finished: 10599194 -> 2115313
1. Converting scanned results into one-hot encoded dataframe.


  0%|          | 0/18656 [00:00<?, ?it/s]

2. Converting results into dictionaries.


  0%|          | 0/16915 [00:00<?, ?it/s]

  0%|          | 0/1093 [00:00<?, ?it/s]

In [ ]:
df = tfi.to_dataframe()
df.head()


,peak_id,gene_short_name,9430076C15RIK,AC002126.6,AC012531.1,AC226150.2,AFP,AHR,AHRR,AIRE,...,ZNF784,ZNF8,ZNF816,ZNF85,ZSCAN10,ZSCAN16,ZSCAN22,ZSCAN26,ZSCAN31,ZSCAN4
0,chr10_100009458_100010353,DNMBP,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr10_100185609_100186527,ERLIN1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr10_100229090_100229936,CHUK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,chr10_100267214_100268053,CWF19L1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,chr10_100286203_100287049,BLOC1S2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = tfi.to_dataframe()
parquet_path = celloracle_dir / "tfinfo.parquet"
df.to_parquet(parquet_path)


In [ ]:
from celloracle.utility import save_as_pickled_object

# If you want, you can save the result as a dictionary as follows.
td = tfi.to_dictionary(dictionary_type="TF2targetgenes")
save_as_pickled_object(td, celloracle_dir / "TFinfo_TF2targetgenes.pickled")


In [ ]:
tf_df = df.copy()